In [44]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

#!pip install vitaldb
import vitaldb

df_cases = pd.read_csv("https://api.vitaldb.net/cases")  # clinical information 을 로딩
df_trks = pd.read_csv("https://api.vitaldb.net/trks")  # clinical information 을 로딩


#df_trkData = pd.read_csv("https://api.vitaldb.net/{fd869e25ba82a66cc95b38ed47110bf4f14bb368}")

# asa 컬럼: asa class
# emop 컬럼: 응급 수술 여부
# death_inhosp: 원내 사망 여부
df_trks

,caseid,tname,tid
0,1,BIS/BIS,fd869e25ba82a66cc95b38ed47110bf4f14bb368
1,1,BIS/EEG1_WAV,0aa685df768489a18a5e9f53af0d83bf60890c73
2,1,BIS/EEG2_WAV,ad13b2c39b19193c8ae4a2de4f8315f18d61a57e
3,1,BIS/EMG,2525603efe18d982764dbca457affe7a45e766a9
4,1,BIS/SEF,1c91aec859304840dec75acf4a35da78be0e8ef0
...,...,...,...
486444,6388,Solar8000/VENT_PIP,2d63adbc7e2653f14348e219816673cde3358cf6
486445,6388,Solar8000/VENT_PPLAT,6f6604255858ddc8f6a01b9f4774b0d43105f6da
486446,6388,Solar8000/VENT_RR,f34f3fae7fd963355c1c7060e1e876d20fa87536
486447,6388,Solar8000/VENT_SET_TV,4a4a55b8aebf9c76a4a76f62a7c1ec6fcb80e618


In [23]:
caseids = list(
    set(df_trks[df_trks['tname'] == 'Solar8000/ART_MBP']['caseid']) & \
#     set(df_cases[df_cases['age'] >= 20]['caseid']) & \
#     set(df_cases[df_cases['age'] < 40]['caseid']) & \
     set(df_cases[df_cases['sex'] =='M']['caseid']) & \
    set(df_cases[~df_cases['opname'].str.contains("transplant")]['caseid'])
)
print('Total {} cases found'.format(len(caseids)))
np.random.shuffle(caseids)  # caseid를 무작위로 섞음

Total 1938 cases found


In [14]:
df_trks[df_trks['tname'] == 'Solar8000/ART_MBP']

,caseid,tname,tid
45,1,Solar8000/ART_MBP,5e68ffda69677a28dca49f361e088903239fd1cf
202,3,Solar8000/ART_MBP,ee8c1ca0a249033e51930ced23811a6eb03123b2
282,4,Solar8000/ART_MBP,4d07a93148cf8a3ee8b4da5d5f718a1adb4d3264
524,7,Solar8000/ART_MBP,30a4c59beb2c29881c969ff720e489aba3465c30
757,10,Solar8000/ART_MBP,ec78a157a94fb779e0f1e4455e5bad3035b89e4c
...,...,...,...
486044,6383,Solar8000/ART_MBP,b77c7d47f3b58eb22899652bb496b99d43292f6e
486194,6385,Solar8000/ART_MBP,24e88aa01145c5df270537ebf15d3df4489dfbbc
486269,6386,Solar8000/ART_MBP,e153e749770205677568e74e7d168bac27c89db9
486345,6387,Solar8000/ART_MBP,0b7af12d8b16d6ef38fb1679e5a6216e30b356fd


In [9]:
import vitaldb
import pandas as pd
caseid = 93
#mbps = vitaldb.load_case(caseid, ['Solar8000/ART_MBP','Solar8000/ART_SBP','Solar8000/ART_DBP' ], 2)
mbps = vitaldb.load_case(caseid, ['SNUADC/ART'])
# plt.axhline(y=65, color='black', linewidth=1)
#plt.figure(figsize=(20, 5))
#plt.plot(mbps)
a = pd.DataFrame(mbps)
a.to_csv("art_test.csv")

In [50]:
import matplotlib.pyplot as plt

from vitaldb import load_case

cases = load_case('SNUADC/ART,Solar8000/ART_SBP', interval=1/100)


for caseid, case in cases.items():

    plt.figure(figsize=(20, 5))

    plt.plot(case[:, 0])

    plt.savefig('{}.png'.format(caseid))

TypeError: load_case() missing 1 required positional argument: 'tnames'

In [53]:
import pandas as pd
import numpy as np
import vitaldb
import os

#import matplotlib.pyplot as plt

df_trks = pd.read_csv("https://api.vitaldb.net/trks")

# find cases which have both ART_MBP and SNUADC/ART
caseids = list(
    set(df_trks[df_trks['tname'] == 'Solar8000/ART_SBP']['caseid']) & 
    set(df_trks[df_trks['tname'] == 'Solar8000/ART_DBP']['caseid']) & 
    set(df_trks[df_trks['tname'] == 'Solar8000/ART_MBP']['caseid']) & 
    set(df_trks[df_trks['tname'] == 'SNUADC/ART']['caseid'])
)

all_sbps = []
all_dbps = []
all_mbps = []
all_art_sbps = []
all_art_dbps = []
all_art_mbps = []

output = 'caseid\tlen\tsbp\tsbp%\tdbp\tdbp%\tmbp\tmbp%\n'
print(output, end='')

fo = open('qa.tsv', 'w')
fo.write(output)

for caseid in caseids:
    if caseid < 4640:
        continue
        
    sbp_tid = df_trks[(df_trks['caseid'] == caseid) & (df_trks['tname'] == 'Solar8000/ART_SBP')]['tid'].values[0]
    dbp_tid = df_trks[(df_trks['caseid'] == caseid) & (df_trks['tname'] == 'Solar8000/ART_DBP')]['tid'].values[0]
    mbp_tid = df_trks[(df_trks['caseid'] == caseid) & (df_trks['tname'] == 'Solar8000/ART_MBP')]['tid'].values[0]
    art_tid = df_trks[(df_trks['caseid'] == caseid) & (df_trks['tname'] == 'SNUADC/ART')]['tid'].values[0]

    SRATE = 100
    SEGLEN = 30
    vals = vitaldb.load_trks([sbp_tid, dbp_tid, mbp_tid, art_tid], 1 / SRATE)

    # extract segements
    case_sbps = []
    case_dbps = []
    case_mbps = []
    case_art_sbps = []
    case_art_dbps = []
    case_art_mbps = []

    raw_sbps_list = []
    raw_dbps_list = []
    raw_mbps_list = []
    raw_arts_list = []
    for i in range(0, len(vals) - SEGLEN * SRATE, SEGLEN * SRATE):
        raw_sbps = vals[i:i + SRATE * SEGLEN, 0]
        raw_dbps = vals[i:i + SRATE * SEGLEN, 1]
        raw_mbps = vals[i:i + SRATE * SEGLEN, 2]
        raw_arts = vals[i:i + SRATE * SEGLEN, 3]

        sbps = raw_sbps[~np.isnan(raw_sbps)]
        dbps = raw_dbps[~np.isnan(raw_dbps)]
        mbps = raw_mbps[~np.isnan(raw_mbps)]
        arts = raw_arts[~np.isnan(raw_arts)]

        # exclude artifacts
        if len(sbps) < SEGLEN * 0.5 * 0.9:
            #print('sbp nan > 10%')
            continue
        elif len(dbps) < SEGLEN * 0.5 * 0.9:
            #print('dbp nan > 10%')
            continue
        elif len(mbps) < SEGLEN * 0.5 * 0.9:
            #print('mbp nan > 10%')
            continue
        elif len(arts) < SEGLEN * SRATE * 0.9:
            #print('art nan > 10%')
            continue
        elif np.max(mbps) > 200 or np.min(mbps) < 30:
            #print('mbp max > 200 or mbp min < 30')
            continue
        elif np.max(arts) > 200 or np.min(arts) < 30:
            #print('art max > 200 or art min < 30')
            continue
        elif (np.abs(np.diff(mbps)) > 10).any():
            #print('abs diff > 10')
            continue

        raw_sbps_list.append(raw_sbps)
        raw_dbps_list.append(raw_dbps)
        raw_mbps_list.append(raw_mbps)
        raw_arts_list.append(raw_arts)

        case_sbps.append(np.max(sbps))
        case_dbps.append(np.min(dbps))
        case_mbps.append(np.mean(mbps))
        case_art_sbps.append(np.max(arts))
        case_art_dbps.append(np.min(arts))
        case_art_mbps.append(np.mean(arts))

    if len(case_sbps) < 10:
        continue
    elif len(case_dbps) < 10:
        continue
    elif len(case_mbps) < 10:
        continue
    
    all_sbps.extend(case_sbps)
    all_dbps.extend(case_dbps)
    all_mbps.extend(case_mbps)
    all_art_sbps.extend(case_art_sbps)
    all_art_dbps.extend(case_art_dbps)
    all_art_mbps.extend(case_art_mbps)

    err_sbp = np.mean(np.abs(np.array(case_sbps) - np.array(case_art_sbps)))
    err_dbp = np.mean(np.abs(np.array(case_dbps) - np.array(case_art_dbps)))
    err_mbp = np.mean(np.abs(np.array(case_mbps) - np.array(case_art_mbps)))
    err_sbp_perc = 100 * err_sbp / np.mean(case_sbps)
    err_dbp_perc = 100 * err_dbp / np.mean(case_dbps)
    err_mbp_perc = 100 * err_mbp / np.mean(case_mbps)

    # if err_sbp_perc > 10 or err_dbp_perc > 10 or err_mbp_perc > 10:
    #     imgdir = 'imgs/' + str(caseid)
    #     if not os.path.exists(imgdir):
    #         os.makedirs(imgdir)

    #     for i in range(len(raw_sbps_list)):
    #         plt.figure(figsize=(20,5))
    #         plt.plot(raw_sbps_list[i], 'go')
    #         plt.plot(raw_dbps_list[i], 'go')
    #         plt.plot(raw_mbps_list[i], 'go')
    #         plt.plot(raw_arts_list[i], 'r-')
    #         plt.savefig(imgdir + "/" + str(i))
    #         plt.close()

    output = '{}\t{}\t{:.1f}\t{:.1f}%\t{:.1f}\t{:.1f}%\t{:.1f}\t{:.1f}%\n'.format(
        caseid, len(case_sbps), err_sbp, err_sbp_perc, err_dbp, err_dbp_perc, err_mbp, err_mbp_perc)
    fo.write(output)
    print(output, end='')

fo.close()

caseid	len	sbp	sbp%	dbp	dbp%	mbp	mbp%


AttributeError: module 'vitaldb' has no attribute 'load_trks'